# CNN for Rare Event Binary Classification


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from pylab import rcParams

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score

from sklearn.model_selection import train_test_split

SEED = 123 
DATA_SPLIT_PCT = 0.2

rcParams['figure.figsize'] = 8, 6
LABELS = ["Normal","Break"]

In [4]:
df=pd.read_csv("APS_selected_without_null.csv")
df

,Unnamed: 0,bj_000,ci_000,ay_002,ag_001,ay_005,ag_002,am_0,ay_006,ee_003,...,cs_002,aq_000,bf_000,at_000,ee_005,bb_000,bl_000,ee_000,bo_000,bk_000
0,0,799478.0,5245752.00,2198.356,828.488,469014.000000,8571.650,107278.622,4.239660e+06,493384.0,...,267998.0,1132040.0,50.000000,6709.982,469792.0,6700214.0,353400.000000,965866.0,283680.00,330760.00
1,1,392208.0,2291079.36,0.000,0.000,71510.000000,0.000,0.000,7.727200e+05,178064.0,...,164860.0,338544.0,350.000000,0.000,245416.0,3646660.0,359780.000000,664504.0,189120.00,341420.00
2,2,139730.0,2322692.16,0.000,0.000,202696.500000,0.000,486.750,8.704560e+05,159812.0,...,56362.0,153698.0,4.000000,0.000,409564.0,2673338.0,130640.000000,262032.0,106380.00,137160.00
3,3,3090.0,2135.04,0.000,0.000,181363.333333,0.000,3894.000,1.781648e+06,46.0,...,24.0,1014.0,107.933333,0.000,44.0,21614.0,253274.666667,5670.0,75648.00,246502.00
4,4,399410.0,3565684.80,0.000,0.000,372236.000000,0.000,811.250,2.128914e+06,229790.0,...,80950.0,551022.0,2.000000,0.000,347188.0,4289260.0,282560.000000,404740.0,59100.00,306780.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,59995,678792.0,8289544.32,5757.992,1061.362,45114.000000,6691.674,2114.000,2.533260e+06,566884.0,...,138036.0,776370.0,14.000000,1574.588,1218244.0,10845228.0,221620.000000,1608808.0,152420.00,263320.00
59996,59996,9572.0,123344.64,5757.992,1061.362,4610.000000,6691.674,76769.085,9.912000e+04,6760.0,...,3606.0,11622.0,66.645000,1574.588,68424.0,147638.0,163342.460000,13934.0,99662.58,160036.47
59997,59997,6224.0,7518.72,5757.992,1061.362,137882.849000,6691.674,76768.344,1.062160e+06,386.0,...,72.0,874.0,66.671000,1603.238,144.0,43738.0,163508.610000,15876.0,99843.76,160040.61
59998,59998,1032454.0,4776505.92,5757.992,1061.362,119742.000000,6691.558,76758.797,4.445146e+06,222654.0,...,333922.0,1214970.0,70.000000,57300.000,225724.0,6159728.0,332300.000000,1180714.0,362360.00,302700.00


In [5]:
df = df.drop(['Unnamed: 0'], axis=1)

In [7]:
df1 = pd.read_csv('aps_failure_training_set.csv')
df1
df1['class'] = df1['class'].replace({'pos': 1, 'neg': 0})
df1
y = df1['class']

y=y.fillna(0)
y

0        0
1        0
2        0
3        0
4        0
        ..
59995    0
59996    0
59997    0
59998    0
59999    0
Name: class, Length: 60000, dtype: int64

# Prepare data for CNN models

In [8]:
input_X = df.values  # converts the df to a numpy array
input_y = df1['class'].values

n_features = input_X.shape[1]  # number of features

In [9]:
n_features

58

In [10]:
input_y

array([0, 0, 0, ..., 0, 0, 0])

In [11]:
input_X

array([[7.99478000e+05, 5.24575200e+06, 2.19835600e+03, ...,
        9.65866000e+05, 2.83680000e+05, 3.30760000e+05],
       [3.92208000e+05, 2.29107936e+06, 0.00000000e+00, ...,
        6.64504000e+05, 1.89120000e+05, 3.41420000e+05],
       [1.39730000e+05, 2.32269216e+06, 0.00000000e+00, ...,
        2.62032000e+05, 1.06380000e+05, 1.37160000e+05],
       ...,
       [6.22400000e+03, 7.51872000e+03, 5.75799200e+03, ...,
        1.58760000e+04, 9.98437600e+04, 1.60040610e+05],
       [1.03245400e+06, 4.77650592e+06, 5.75799200e+03, ...,
        1.18071400e+06, 3.62360000e+05, 3.02700000e+05],
       [2.60378000e+05, 2.48011392e+06, 0.00000000e+00, ...,
        4.09798000e+05, 0.00000000e+00, 1.91320000e+05]])

In [12]:
n_features

58

### Divide the data into train, valid, and test

In [14]:
X_train, X_test, y_train, y_test = train_test_split(np.array(input_X), np.array(input_y), test_size=DATA_SPLIT_PCT, random_state=SEED)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=DATA_SPLIT_PCT, random_state=SEED)


In [15]:
X_train.shape

(38400, 58)

In [16]:
X_train

array([[1.09940000e+04, 3.05625600e+04, 2.08080000e+03, ...,
        5.23600000e+04, 1.21010100e+05, 1.74791690e+05],
       [2.07842000e+05, 2.61103488e+06, 7.88211600e+03, ...,
        2.88932000e+05, 9.25348800e+04, 1.19120000e+05],
       [2.73080000e+04, 4.30803840e+05, 2.80211200e+03, ...,
        5.48200000e+04, 1.20251340e+05, 1.69119790e+05],
       ...,
       [5.53246000e+05, 2.39988768e+06, 2.38472000e+03, ...,
        7.52664000e+05, 1.16885920e+05, 2.67480000e+05],
       [6.86122000e+05, 2.81835840e+06, 2.54958000e+02, ...,
        5.39352000e+05, 1.13897500e+05, 4.11400000e+05],
       [6.59054000e+05, 9.60110784e+06, 8.75752000e+02, ...,
        1.55190400e+06, 1.38300000e+05, 1.15780000e+05]])

In [17]:
np.array(input_y)

array([0, 0, 0, ..., 0, 0, 0])

In [18]:
X_train_y0 = X_train[y_train==0]
X_train_y1 = X_train[y_train==1]

X_valid_y0 = X_valid[y_valid==0]
X_valid_y1 = X_valid[y_valid==1]

In [19]:
X_train_y0.shape

(37770, 58)

In [20]:
X_valid_y0.shape

(9427, 58)

In [21]:
len(X_train)

38400

In [22]:
num_classes=2

In [23]:
# Verify the reshaped data shape
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)


X_train shape: (38400, 58)
X_test shape: (12000, 58)


In [24]:
# Reshape the data to match the expected input shape of the model
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


In [25]:
# Verify the reshaped data shape
print("X_train shape:", X_train_reshaped.shape)
print("X_test shape:", X_test_reshaped.shape)


X_train shape: (38400, 58, 1)
X_test shape: (12000, 58, 1)


In [36]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense


# Define the model
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))



In [37]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [38]:
# Train the model
model.fit(X_train_reshaped, y_train, epochs=100, batch_size=32, validation_data=(X_test_reshaped, y_test))

Epoch 1/100
23675/23675 [==============================] - 121s 5ms/step - loss: 0.0369 - accuracy: 0.9942 - val_loss: 0.0352 - val_accuracy: 0.9943
Epoch 2/100
23675/23675 [==============================] - 119s 5ms/step - loss: 0.0362 - accuracy: 0.9942 - val_loss: 0.0355 - val_accuracy: 0.9943
Epoch 3/100
23675/23675 [==============================] - 117s 5ms/step - loss: 0.0360 - accuracy: 0.9942 - val_loss: 0.0350 - val_accuracy: 0.9943
Epoch 4/100
23675/23675 [==============================] - 119s 5ms/step - loss: 0.0359 - accuracy: 0.9942 - val_loss: 0.0350 - val_accuracy: 0.9943
Epoch 5/100
23675/23675 [==============================] - 129s 5ms/step - loss: 0.0360 - accuracy: 0.9942 - val_loss: 0.0350 - val_accuracy: 0.9943
Epoch 6/100
23675/23675 [==============================] - 129s 5ms/step - loss: 0.0360 - accuracy: 0.9942 - val_loss: 0.0349 - val_accuracy: 0.9943
Epoch 7/100
23675/23675 [==============================] - 129s 5ms/step - loss: 0.0360 - accuracy: 0.9942

In [39]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_reshaped, y_test, verbose=0)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


Test Loss: 0.03499554470181465
Test Accuracy: 0.9943400025367737


In [40]:
# Make predictions
predictions = model.predict(X_test_reshaped)

7399/7399 [==============================] - 14s 2ms/step


In [41]:
predictions

array([[0.00484208],
       [0.00484208],
       [0.00484208],
       ...,
       [0.00484208],
       [0.00484208],
       [0.00484208]], dtype=float32)

In [42]:
predicted_labels = np.round(predictions)  # Round the predictions to obtain binary labels

In [43]:
# Print confusion matrix and classification report
cm = confusion_matrix(y_test, predicted_labels)
print('Confusion Matrix:')
print(cm)


Confusion Matrix:
[[235410      0]
 [  1340      0]]


In [44]:
print('Classification Report:')
print(classification_report(y_test, predicted_labels))

Classification Report:


/Users/chathurangishyalika/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00    235410
         1.0       0.00      0.00      0.00      1340

    accuracy                           0.99    236750
   macro avg       0.50      0.50      0.50    236750
weighted avg       0.99      0.99      0.99    236750



/Users/chathurangishyalika/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/chathurangishyalika/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
